In [4]:
import fitz  # PyMuPDF
import requests
import os
import logging
from pathlib import Path
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup

doc_id = "AML-1"

# --- CONFIGURATION ---
# 1. 您要分析的來源 PDF 檔案路徑
SOURCE_PDF_PATH = Path(f"C:/Users/User/Desktop/Microsoft_GraphRAG/UnstructuredData_Transformation_Pipeline/SPM/{doc_id}.pdf")

# 2. 下載的 PDF 要儲存的資料夾
DOWNLOAD_DIR = Path(f"C:/Users/User/Desktop/Microsoft_GraphRAG/UnstructuredData_Transformation_Pipeline/SPM/cross_reference/{doc_id}")

# 3. 日誌檔案的名稱
LOG_FILE = f"{doc_id}_hyperlink_download.log"

# 4. 用於解析重新導向頁面中的相對路徑 (如果需要)
# 例如，如果 HTML 頁面中的連結是 /path/to/file.pdf，它會與此域名結合
BASE_DOMAIN_FOR_REDIRECT = "https://brdr.hkma.gov.hk"
# --- END OF CONFIGURATION ---


def setup_logging(log_file_path: Path):
    """設定日誌記錄器，同時輸出到控制台和檔案"""
    log_file_path.parent.mkdir(parents=True, exist_ok=True)
    
    logger = logging.getLogger("hyperlink_downloader")
    logger.setLevel(logging.INFO)

    if logger.hasHandlers():
        logger.handlers.clear()

    file_handler = logging.FileHandler(log_file_path, encoding='utf-8', mode='w')
    file_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(file_formatter)
    logger.addHandler(file_handler)

    console_handler = logging.StreamHandler()
    console_formatter = logging.Formatter('%(levelname)s: %(message)s')
    console_handler.setFormatter(console_formatter)
    logger.addHandler(console_handler)
    
    return logger

def extract_hyperlinks_from_pdf(pdf_path: Path, logger: logging.Logger) -> set:
    """從 PDF 檔案中提取所有內嵌超連結的 URI"""
    if not pdf_path.exists():
        logger.error(f"來源 PDF 檔案不存在: {pdf_path}")
        return set()
    
    unique_uris = set()
    try:
        logger.info(f"正在從 {pdf_path.name} 提取內嵌超連結...")
        doc = fitz.open(pdf_path)
        for page_num, page in enumerate(doc):
            links = page.get_links()
            for link in links:
                if "uri" in link and link["uri"]:
                    unique_uris.add(link["uri"])
        doc.close()
        
        if unique_uris:
            logger.info(f"在文件中找到 {len(unique_uris)} 個不重複的超連結 URI。")
        else:
            logger.warning("在文件中未找到任何內嵌超連結。")
            
        return unique_uris
    except Exception as e:
        logger.error(f"讀取或解析 PDF 時發生錯誤: {e}")
        return set()

def download_file_from_uri(url: str, download_dir: Path, logger: logging.Logger):
    """從給定的 URI 下載檔案，並能處理重新導向到 HTML 頁面的情況"""
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        from requests.packages.urllib3.exceptions import InsecureRequestWarning
        requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

        logger.info(f"正在處理 URI: {url}")
        response = requests.get(url, timeout=30, verify=False, headers=headers)
        response.raise_for_status()

        content_type = response.headers.get('Content-Type', '')
        
        # 情況 1: 連結直接回傳 PDF
        if 'application/pdf' in content_type:
            pdf_url = url
            pdf_response = response
        # 情況 2: 連結重新導向到一個 HTML 頁面
        elif 'text/html' in content_type:
            logger.info("連結指向一個 HTML 頁面，正在嘗試從中尋找真正的 PDF 連結...")
            soup = BeautifulSoup(response.content, 'html.parser')
            pdf_link_tag = soup.find('a', href=lambda href: href and href.lower().endswith('.pdf'))
            
            if not pdf_link_tag:
                logger.warning(f"在 HTML 頁面 {url} 中未找到 PDF 下載連結。")
                return
            
            pdf_relative_path = pdf_link_tag['href']
            pdf_url = urljoin(BASE_DOMAIN_FOR_REDIRECT, pdf_relative_path)
            logger.info(f"找到真正的 PDF 連結: {pdf_url}")
            
            # 重新下載真正的 PDF
            pdf_response = requests.get(pdf_url, stream=True, timeout=30, verify=False, headers=headers)
            pdf_response.raise_for_status()
        else:
            logger.warning(f"連結 {url} 的內容類型不是 PDF 或 HTML ({content_type})，跳過。")
            return

        # 執行下載
        filename = os.path.basename(urlparse(pdf_url).path)
        save_path = download_dir / filename
        
        if save_path.exists():
            logger.info(f"檔案已存在，跳過下載: {filename}")
            return

        logger.info(f"正在下載檔案: {filename}")
        with open(save_path, 'wb') as f:
            # 使用 .raw.read() 確保下載完整性
            f.write(pdf_response.content)
        logger.info(f"✅ 下載成功: {filename}")

    except requests.exceptions.HTTPError as e:
        logger.error(f"❌ 下載失敗 (HTTP 錯誤 {e.response.status_code}): {url}")
    except requests.exceptions.RequestException as e:
        logger.error(f"❌ 下載失敗 (網路錯誤): {url} - {e}")
    except Exception as e:
        logger.error(f"❌ 處理 URI 時發生未知錯誤: {url} - {e}")


def main():
    """主執行函式"""
    logger = setup_logging(DOWNLOAD_DIR / LOG_FILE)
    logger.info("--- 超連結提取與下載任務開始 ---")
    
    DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)
    
    # 1. 從 PDF 提取超連結
    uris_to_download = extract_hyperlinks_from_pdf(SOURCE_PDF_PATH, logger)
    
    if not uris_to_download:
        logger.info("任務結束，沒有找到可處理的超連結。")
        return

    # 2. 篩選出包含 .pdf 的連結並下載
    pdf_uris = {uri for uri in uris_to_download if '.pdf' in uri.lower()}
    logger.info(f"篩選出 {len(pdf_uris)} 個指向 PDF 的連結準備下載。")

    for uri in sorted(list(pdf_uris)):
        download_file_from_uri(uri, DOWNLOAD_DIR, logger)
        
    logger.info("--- 所有下載任務已完成 ---")


if __name__ == "__main__":
    main()

INFO: --- 超連結提取與下載任務開始 ---
INFO: 正在從 AML-1.pdf 提取內嵌超連結...
INFO: 在文件中找到 4 個不重複的超連結 URI。
INFO: 篩選出 2 個指向 PDF 的連結準備下載。
INFO: 正在處理 URI: http://www.hkma.gov.hk/media/eng/doc/key-functions/banking-stability/supervisory-policy-manual/GL.pdf
INFO: 連結指向一個 HTML 頁面，正在嘗試從中尋找真正的 PDF 連結...
INFO: 找到真正的 PDF 連結: https://brdr.hkma.gov.hk/eng/doc-ldg/docId/getPdf/20250410-1-EN/GL.pdf
INFO: 正在下載檔案: GL.pdf
INFO: ✅ 下載成功: GL.pdf
INFO: 正在處理 URI: http://www.hkma.gov.hk/media/eng/doc/key-functions/banking-stability/supervisory-policy-manual/IN.pdf
INFO: 連結指向一個 HTML 頁面，正在嘗試從中尋找真正的 PDF 連結...
INFO: 找到真正的 PDF 連結: https://brdr.hkma.gov.hk/eng/doc-ldg/docId/getPdf/20250213-1-EN/IN.pdf
INFO: 正在下載檔案: IN.pdf
INFO: ✅ 下載成功: IN.pdf
INFO: --- 所有下載任務已完成 ---
